In [0]:
%pip install scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.0 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python
     

In [0]:
from scapy.all import rdpcap, IP, TCP

# Load the capture
packets = rdpcap("/Volumes/pcap/default/http/http.cap")

# Analyze first 5 packets
for i, pkt in enumerate(packets[:5]):
    print(f"Packet {i+1}")
    print(f"  Length: {len(pkt)}")
    
    # Check and print layers
    if IP in pkt:
        print("  Transport Layer: IP")
    if TCP in pkt:
        print("  Transport Layer: TCP")
    if pkt.haslayer('Raw') and b"HTTP" in bytes(pkt['Raw']):
        print("  Application Layer: HTTP")
    
    print(f"  Summary: {pkt.summary()}")
    print("-" * 40)

Packet 1
  Length: 62
  Transport Layer: IP
  Transport Layer: TCP
  Summary: Ether / IP / TCP 145.254.160.237:3372 > 65.208.228.223:http S
----------------------------------------
Packet 2
  Length: 62
  Transport Layer: IP
  Transport Layer: TCP
  Summary: Ether / IP / TCP 65.208.228.223:http > 145.254.160.237:3372 SA
----------------------------------------
Packet 3
  Length: 54
  Transport Layer: IP
  Transport Layer: TCP
  Summary: Ether / IP / TCP 145.254.160.237:3372 > 65.208.228.223:http A
----------------------------------------
Packet 4
  Length: 533
  Transport Layer: IP
  Transport Layer: TCP
  Application Layer: HTTP
  Summary: Ether / IP / TCP 145.254.160.237:3372 > 65.208.228.223:http PA / Raw
----------------------------------------
Packet 5
  Length: 54
  Transport Layer: IP
  Transport Layer: TCP
  Summary: Ether / IP / TCP 65.208.228.223:http > 145.254.160.237:3372 A
----------------------------------------


In [0]:
from collections import Counter
from scapy.all import rdpcap, IP

packets = rdpcap("/Volumes/pcap/default/http/http.cap")

ip_counts = Counter()

for pkt in packets:
    if IP in pkt:
        ip_counts[pkt[IP].src] += 1

print("Top 5 Source IPs:")
for ip, count in ip_counts.most_common(5):
    print(f"{ip}: {count} packets")

Top 5 Source IPs:
145.254.160.237: 20 packets
65.208.228.223: 18 packets
216.239.59.99: 4 packets
145.253.2.203: 1 packets


In [0]:
from scapy.all import rdpcap, TCP

packets = rdpcap("/Volumes/pcap/default/http/http.cap")

print("Visited URLs:")
for pkt in packets:
    if pkt.haslayer(TCP) and pkt.haslayer('Raw'):
        payload = pkt['Raw'].load
        if b"GET" in payload or b"POST" in payload:
            try:
                lines = payload.split(b"\r\n")
                host_line = next(line for line in lines if b"Host:" in line)
                uri_line = lines[0]
                host = host_line.split(b":")[1].strip().decode()
                uri = uri_line.split(b" ")[1].decode()
                print(f"http://{host}{uri}")
            except:
                continue

Visited URLs:
http://www.ethereal.com/download.html
http://pagead2.googlesyndication.com/pagead/ads?client=ca-pub-2309191948673629&random=1084443430285&lmt=1082467020&format=468x60_as&output=html&url=http%3A%2F%2Fwww.ethereal.com%2Fdownload.html&color_bg=FFFFFF&color_text=333333&color_link=000000&color_url=666633&color_border=666633


In [0]:
from scapy.all import rdpcap, DNS, DNSQR
from collections import Counter

# Load the capture file
packets = rdpcap("/Volumes/pcap/default/http/dns.cap")

# Counter for domain names
domain_counter = Counter()

# Iterate over packets
for pkt in packets:
    if pkt.haslayer(DNS) and pkt.getlayer(DNS).qr == 0:  # qr=0 means it's a query
        query = pkt.getlayer(DNSQR)
        if query:
            domain = query.qname.decode().strip('.')
            domain_counter[domain] += 1

# Print top 10 queried domains
print("Top 10 Queried Domains:")
for domain, count in domain_counter.most_common(10):
    print(f"{domain}: {count} times")

Top 10 Queried Domains:
google.com: 3 times
www.netbsd.org: 3 times
GRIMM.utelsystems.local: 2 times
104.9.192.66.in-addr.arpa: 1 times
www.google.com: 1 times
www.l.google.com: 1 times
www.example.com: 1 times
www.example.notginh: 1 times
www.isc.org: 1 times
1.0.0.127.in-addr.arpa: 1 times
